In [ ]:
import numpy as np
import holoviews as hv
from holoviews import opts
import xarray as xr
import hvplot.xarray

In [ ]:
import xarray as xr
import xrspatial as xrs
import datashader as ds
from datashader.transfer_functions import shade
from datashader.transfer_functions import stack
import geoviews as gv
import rioxarray as rx
from math import isnan

In [ ]:
#help(xrs.where())

In [ ]:
data = rx.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif",masked =True).squeeze().astype('float64')

In [ ]:
data

In [ ]:
data = data.where(data>0)

In [ ]:
new=data.where(np.logical_and(data>18,data<295))

In [ ]:
int((~np.isnan(new.values)).sum())

In [ ]:
def getpercentage(orig,other):
    origcount = int((~np.isnan(orig.values)).sum())
    newcount = int((~np.isnan(other.values)).sum())
    return round((origcount-newcount)/origcount*100)

In [ ]:
(int((~np.isnan(data.values)).sum())-int((~np.isnan(new.values)).sum()))/int((~np.isnan(data.values)).sum())

In [ ]:
getpercentage(data,new)